# 触媒促進剤除外

In [121]:
import pandas as pd
df = pd.read_csv('1121.csv',header=0).iloc[:,1:]

In [122]:
df.head(5)

,entry,R1-,organocatalyst,organocatalyst(mol%),temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),収率(%),R2_C,R2_H,R2_Ph,under_Ar,under_O2,under_air
0,1,CC1=CC=CC=C1,O=C1C(C(C)(C)C)=CC(C(O)=C1)=O,1.5,23,20.0,0.0,0.0,87.0,0,1,0,0,1,0
1,2,CC1=CC=C(N)C=C1,O=C1C(C(C)(C)C)=CC(C(O)=C1)=O,1.5,23,20.0,0.0,0.0,76.0,0,1,0,0,1,0
2,3,CC1=CC=C(OC)C=C1,O=C1C(C(C)(C)C)=CC(C(O)=C1)=O,1.5,23,20.0,0.0,0.0,93.0,0,1,0,0,1,0
3,4,CC1=CC=C(Cl)C=C1,O=C1C(C(C)(C)C)=CC(C(O)=C1)=O,1.5,23,20.0,0.0,0.0,90.0,0,1,0,0,1,0
4,5,CC1=CC=C(F)C=C1,O=C1C(C(C)(C)C)=CC(C(O)=C1)=O,1.5,23,20.0,0.0,0.0,91.0,0,1,0,0,1,0


In [125]:
check_columns = df_removed.columns.drop(['収率(%)'])

# 説明変数のみでみた時の重複削除
_df = df_removed.drop_duplicates(subset=check_columns).reset_index(drop=True)
X = _df.drop(columns=['収率(%)'])
y = _df['収率(%)']

In [126]:
df_removed[df_removed.duplicated(subset=check_columns,keep=False)]

,entry,R1-,organocatalyst,organocatalyst(mol%),temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),収率(%),R2_C,R2_H,R2_Ph,under_Ar,under_O2,under_air


In [127]:
X.columns

Index(['entry', 'R1-', 'organocatalyst', 'organocatalyst(mol%)', 'temp(℃)',
       'time(h)', 'Cu(OAc)2(mol%)', 'AcOH(mol%)', 'R2_C', 'R2_H', 'R2_Ph',
       'under_Ar', 'under_O2', 'under_air'],
      dtype='object')

In [128]:
X.drop(columns=['Cu(OAc)2(mol%)','AcOH(mol%)'],inplace=True)

In [129]:
# 説明変数をスケーリング
# 目的変数は必要ない。
# https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re
from sklearn.preprocessing import StandardScaler
scaling_columns = ['organocatalyst(mol%)','temp(℃)','time(h)']
scaler = StandardScaler()
scaler.fit(X[scaling_columns])
X_scaled = pd.concat([X.drop(columns=scaling_columns), 
                           pd.DataFrame(scaler.transform(X[scaling_columns]),index=X.index , columns=scaling_columns)],
                           axis=1, join='inner')

print(X_scaled.shape)

(126, 12)


In [130]:
BIT = 2048

In [131]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

def toFinger(df, columns):
    df_copy = df.drop(columns=columns)
    for column in columns:
        fingerprints = toFingerFromSmiles(df[column])
        column_names = list(map(lambda x: str(x)+'_'+column, range(BIT)))
        df_copy = pd.merge(df_copy, pd.DataFrame(fingerprints,index=df.index, columns=column_names), left_index=True, right_index=True)
    return df_copy

def toFingerFromSmiles(series):
    mols = []
    for smile in series:
        if smile in ['-', 0]: smile = '' 
        mols.append(Chem.MolFromSmiles(smile))

    fingerprints = []
    for mol_idx, mol in enumerate(mols):
        try:
            # listに直してる。
            fingerprint = [x for x in Chem.AllChem.GetMorganFingerprintAsBitVect(mol,2,BIT)]
            fingerprints.append(fingerprint)
        except Exception as e:
            print("Error", mol_idx)
            break
    return fingerprints

In [132]:
# 1. 全てのデータを使用
finger_columns = ['R1-', 'organocatalyst']
X_1 = toFinger(X_scaled, finger_columns)
y_1 = y

In [135]:
X_1.to_csv('X_removed_promoter.csv')

In [136]:
y.to_csv('y_removed_promoter.csv')

In [140]:
X_removed_promoter_and_others = X_1.drop(columns=['organocatalyst(mol%)','temp(℃)', 'time(h)'])

In [143]:
X_removed_promoter_and_others.duplicated().value_counts()

False    126
dtype: int64

In [144]:
X_removed_promoter_and_others.to_csv('X_removed_promoter_and_others.csv')

# R1-がCC1=CC=CC=C1

In [160]:
dataset = pd.concat([X_scaled,y],axis=1)
dataset.head(5)

,entry,R1-,organocatalyst,R2_C,R2_H,R2_Ph,under_Ar,under_O2,under_air,organocatalyst(mol%),temp(℃),time(h),収率(%)
0,1,CC1=CC=CC=C1,O=C1C(C(C)(C)C)=CC(C(O)=C1)=O,0,1,0,0,1,0,-1.522195,-0.967349,-0.392607,87.0
1,2,CC1=CC=C(N)C=C1,O=C1C(C(C)(C)C)=CC(C(O)=C1)=O,0,1,0,0,1,0,-1.522195,-0.967349,-0.392607,76.0
2,3,CC1=CC=C(OC)C=C1,O=C1C(C(C)(C)C)=CC(C(O)=C1)=O,0,1,0,0,1,0,-1.522195,-0.967349,-0.392607,93.0
3,4,CC1=CC=C(Cl)C=C1,O=C1C(C(C)(C)C)=CC(C(O)=C1)=O,0,1,0,0,1,0,-1.522195,-0.967349,-0.392607,90.0
4,5,CC1=CC=C(F)C=C1,O=C1C(C(C)(C)C)=CC(C(O)=C1)=O,0,1,0,0,1,0,-1.522195,-0.967349,-0.392607,91.0


In [162]:
dataset_removed_R1 = dataset[dataset['R1-'] == 'CC1=CC=CC=C1'].drop(columns=['R1-'])
X_removed_R1 = dataset_removed_R1.drop(columns=['収率(%)'])
y_removed_R1 = dataset_removed_R1['収率(%)']

In [163]:
finger_columns = ['organocatalyst']
toFinger(X_removed_R1, finger_columns).to_csv('X_removed_R1.csv')
y_removed_R1.to_csv('y_removed_R1.csv')

In [155]:
_d

,entry,R2_C,R2_H,R2_Ph,under_Ar,under_O2,under_air,organocatalyst(mol%),temp(℃),time(h),...,2038_organocatalyst,2039_organocatalyst,2040_organocatalyst,2041_organocatalyst,2042_organocatalyst,2043_organocatalyst,2044_organocatalyst,2045_organocatalyst,2046_organocatalyst,2047_organocatalyst
0,1,0,1,0,0,1,0,-1.522195,-0.967349,-0.392607,...,0,0,0,0,0,0,0,0,0,0
13,1,1,0,0,0,1,0,0.724905,-0.967349,1.059298,...,0,0,0,0,0,0,0,0,0,0
26,1,0,1,0,0,0,1,2.046728,0.803166,1.059298,...,0,0,0,0,0,0,0,0,0,0
29,4,1,0,0,0,0,1,-0.596919,-0.967349,1.059298,...,0,0,0,0,0,0,0,0,0,0
30,5,1,0,0,0,0,1,0.724905,-0.967349,1.059298,...,0,0,0,0,0,0,0,0,0,0
31,6,1,0,0,0,0,1,0.724905,-0.967349,1.059298,...,0,0,0,0,0,0,0,0,0,0
32,7,1,0,0,0,0,1,0.724905,-0.967349,1.059298,...,0,0,0,0,0,0,0,0,0,0
33,8,1,0,0,0,0,1,0.724905,-0.967349,1.059298,...,0,0,0,0,0,0,0,0,0,0
34,9,1,0,0,0,0,1,0.724905,-0.967349,1.059298,...,0,0,0,0,0,0,0,0,0,0
35,10,1,0,0,0,0,1,0.724905,-0.967349,1.059298,...,0,0,0,0,0,0,0,0,0,0


# 8.5ぐらいの重みの分析やる！頑張ろう！日本！！！！！！！